In [2]:
import os
from dotenv import load_dotenv
from typing import TypedDict
from langchain_core.messages import HumanMessage, AIMessage
# The import has been updated to the correct location
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

# Load API keys and set up tracing
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Intro to LangGraph"

# --- External Memory Store ---
# In a real app, this would be a database. We'll use a dictionary to simulate it.
memory_store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in memory_store:
        memory_store[session_id] = ChatMessageHistory()
    return memory_store[session_id]

# --- Define the State ---
# The state is minimal, only holding the session ID and the current input.
class GraphState(TypedDict):
    session_id: str
    input: HumanMessage

# --- Define Nodes and Router ---
def call_llm(state: GraphState):
    session_id = state['session_id']
    user_input = state['input']
    
    # Load history from our external store
    memory = get_session_history(session_id)
    memory.add_message(user_input)
    
    # Invoke the model
    llm = ChatOpenAI(model="gpt-4o")
    response = llm.invoke(memory.messages)
    
    # Save the AI's response back to the external store
    memory.add_message(response)
    print(f"AI: {response.content}")
    return {} # No need to update state, as memory is external

def summarize_conversation(state: GraphState):
    print("\n--- SUMMARIZING CONVERSATION ---")
    session_id = state['session_id']
    memory = get_session_history(session_id)
    
    llm = ChatOpenAI(model="gpt-4o")
    summary_prompt = "Summarize this conversation concisely: " + "\n".join([f"{msg.type}: {msg.content}" for msg in memory.messages])
    summary = llm.invoke(summary_prompt).content
    
    # Replace the history in the external store with the summary
    memory.clear()
    memory.add_message(AIMessage(content=f"Conversation summary: {summary}"))
    print(f"New History: {memory.messages[-1].content}")
    return {}

def should_summarize(state: GraphState):
    # The router checks the length of the *external* memory
    session_id = state['session_id']
    memory = get_session_history(session_id)
    print(f"(Checking memory: {len(memory.messages)} messages)")
    if len(memory.messages) > 4:
        return "summarize_conversation"
    else:
        return END

# --- Build the Graph ---
workflow = StateGraph(GraphState)
workflow.add_node("conversation", call_llm)
workflow.add_node("summarize_conversation", summarize_conversation)
workflow.set_entry_point("conversation")
workflow.add_conditional_edges("conversation", should_summarize)
workflow.add_edge("summarize_conversation", END)

# Compile the graph
app = workflow.compile()

# --- Run a simulation of the chatbot ---
session_id = "my-chat-session-1"

print("--- Turn 1 ---")
app.invoke({"session_id": session_id, "input": HumanMessage(content="Hi! I'm Bob.")})

print("\n--- Turn 2 ---")
app.invoke({"session_id": session_id, "input": HumanMessage(content="My favorite color is blue.")})

print("\n--- Turn 3 (This will trigger summarization) ---")
app.invoke({"session_id": session_id, "input": HumanMessage(content="What did I say my name was?")})

--- Turn 1 ---
AI: Hello, Bob! How can I assist you today?
(Checking memory: 2 messages)

--- Turn 2 ---
AI: That's a great choice! Blue is often associated with calmness and serenity. Do you have a specific shade of blue that you prefer?
(Checking memory: 4 messages)

--- Turn 3 (This will trigger summarization) ---
AI: You mentioned that your name is Bob.
(Checking memory: 6 messages)

--- SUMMARIZING CONVERSATION ---
New History: Conversation summary: Bob introduced himself and shared that his favorite color is blue. The AI acknowledged Bob's name, commented on his color choice, and asked if he prefers a specific shade of blue. Bob then asked the AI to recall his name, which the AI correctly did.


{'session_id': 'my-chat-session-1',
 'input': HumanMessage(content='What did I say my name was?', additional_kwargs={}, response_metadata={})}